## <span style='color:#ff5f27'> 📝 Imports

In [1]:
# !pip install -r requirements.txt --quiet

In [2]:
import pandas as pd
import numpy as np
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

from features.users import generate_users
from features.videos import generate_video_content
from features.interactions import generate_interactions

## <span style='color:#ff5f27'> ⚙️ Data Generation</span>


### <span style='color:#ff5f27'> 👥 Users Data Generation</span>

In [3]:
user_data = generate_users(25_000)

data_users_df = pd.DataFrame(user_data)
data_users_df.head()

,user_id,gender,age,country
0,AO292V,Female,68,Estonia
1,EE477J,Other,20,Panama
2,TU771Q,Female,83,Guam
3,TR773B,Male,37,Guatemala
4,IL923A,Male,45,Aruba


### <span style='color:#ff5f27'> 🎥 Content Data Generation</span>


In [4]:
# Generate data for 25_000 videos
video_data = generate_video_content(25_000, historical=True)

data_video_df = pd.DataFrame(video_data)
data_video_df.head()

,video_id,category,views,likes,video_length,upload_date
0,8SX29N,Education,3730,1403,129,2022-04-27
1,8OJ82V,Cooking,2739,1228,61,2023-02-05
2,5PA54E,Dance,22669,10155,167,2022-08-27
3,0MI84G,Sports,270868,209308,21,2023-10-13
4,4ZY80J,Sports,75019,8839,15,2024-03-16


### <span style='color:#ff5f27'> 🔗 Interactions Generation</span>


In [5]:
num_interactions = 1_000_000

# Generate interactions
interactions = generate_interactions(num_interactions, user_data, video_data)

data_interactions_df = pd.DataFrame(interactions)
data_interactions_df.head()

,interaction_id,user_id,video_id,interaction_type,watch_time
0,0659-48-5477,QE013D,4PV70X,skip,122
1,8684-74-0225,BF245B,6QG89F,skip,30
2,3216-60-6868,XH279C,7BT49F,skip,91
3,5613-63-9674,KA693M,7KK05P,skip,26
4,6335-58-2992,KQ918Z,5DA72W,view,141


## <span style="color:#ff5f27">👮🏻‍♂️ Great Expectations </span>

In [6]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_users_df = ge.from_pandas(data_users_df)

# Initialize the expectation suite
expectation_suite_users = ge_users_df.get_expectation_suite()
expectation_suite_users.expectation_suite_name = "user_data_suite"

# Expectation: Age should be between 0 and 120
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "age", "min_value": 12, "max_value": 100}
    )
)

# Expectations: Columns should not have null values
for column in ge_users_df.columns:
    expectation_suite_users.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Gender should only contain specific values
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={"column": "gender", "value_set": ["Male", "Female", "Other"]}
    )
)

{"meta": {}, "kwargs": {"column": "gender", "value_set": ["Male", "Female", "Other"]}, "expectation_type": "expect_column_distinct_values_to_be_in_set"}

In [7]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_video_df = ge.from_pandas(data_video_df)

# Initialize the expectation suite
expectation_suite_videos = ge_video_df.get_expectation_suite()
expectation_suite_videos.expectation_suite_name = "video_data_suite"

# Expectation: Views, Likes, and Video Length should be non-negative
for column in ["views", "likes", "video_length"]:
    expectation_suite_videos.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_between",
            kwargs={"column": column, "min_value": 0, "max_value": None}
        )
    )

# Expectation: Valid date format for upload_date
expectation_suite_videos.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_dateutil_parseable",
        kwargs={"column": "upload_date"}
    )
)

{"meta": {}, "kwargs": {"column": "upload_date"}, "expectation_type": "expect_column_values_to_be_dateutil_parseable"}

In [8]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_interactions_df = ge.from_pandas(data_interactions_df)

# Initialize the expectation suite
expectation_suite_interactions = ge_interactions_df.get_expectation_suite()
expectation_suite_interactions.expectation_suite_name = "interactions_data_suite"

# Expectations: Non-null values in all columns
for column in ge_interactions_df.columns:
    expectation_suite_interactions.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Validate interaction types
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "interaction_type",
            "value_set": ['like', 'dislike', 'view', 'comment', 'share', 'skip']
        }
    )
)

# Expectation: Positive watch time
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "watch_time", "min_value": 0}
    )
)

{"meta": {}, "kwargs": {"column": "watch_time", "min_value": 0}, "expectation_type": "expect_column_values_to_be_between"}

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [9]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/17565
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🪄 Feature Group Creation </span>


In [10]:
users_fg = fs.get_or_create_feature_group(
    name="users",
    description="Users data.",
    version=1,
    primary_key=["user_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_users,
)

users_fg.insert(data_users_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/737610
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/737610


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████████| Rows 25000/25000 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: users_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/17565/jobs/named/users_1_offline_fg_materialization/executions
Done ✅


In [11]:
videos_fg = fs.get_or_create_feature_group(
    name="videos",
    description="Videos data.",
    version=1,
    primary_key=["video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_videos,
)

videos_fg.insert(data_video_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/735583
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/735583


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████████| Rows 25000/25000 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: videos_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/17565/jobs/named/videos_1_offline_fg_materialization/executions
Done ✅


In [12]:
interactions_fg = fs.get_or_create_feature_group(
    name="interactions",
    description="Interactions data.",
    version=1,
    primary_key=["interaction_id", "user_id", "video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_interactions,
)

interactions_fg.insert(data_interactions_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/736588
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/736588


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████| Rows 1000000/1000000 | Elapsed Time: 01:07 | Remaining Time: 00:00


Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/17565/jobs/named/interactions_1_offline_fg_materialization/executions
Done ✅


## <span style="color:#ff5f27">🪄 Ranking Feature Group </span>


In [13]:
video_interactions_df = pd.merge(
    data_interactions_df, 
    data_video_df, 
    on='video_id', 
    how='inner',
)

ranking_df = pd.merge(
    video_interactions_df, 
    data_users_df, 
    on='user_id', 
    how='inner',
)

ranking_df['label'] = np.where(
    ranking_df.interaction_type.isin(['view', 'like', 'share', 'comment']), 
    1, 
    0,
)

ranking_df.drop(
    ['interaction_id', 'interaction_type', 'watch_time'], 
    axis=1, 
    inplace=True,
)

ranking_df.head()

,user_id,video_id,category,views,likes,video_length,upload_date,gender,age,country,label
0,QE013D,4PV70X,Education,118313,29885,122,2023-09-10,Male,83,Macau SAR China,0
1,QE013D,8OT77Y,Sports,2361,1976,49,2022-05-09,Male,83,Macau SAR China,0
2,QE013D,0CA11G,Technology,91177,50317,201,2022-11-27,Male,83,Macau SAR China,0
3,QE013D,0PO32E,Travel,32638,27587,78,2022-06-22,Male,83,Macau SAR China,1
4,QE013D,7TN04H,Dance,4725,669,15,2022-08-20,Male,83,Macau SAR China,0


In [14]:
ranking_fg = fs.get_or_create_feature_group(
    name="ranking",
    description="Ranking Data.",
    version=1,
    primary_key=["user_id", "video_id"],
    online_enabled=True,
)

ranking_fg.insert(ranking_df)
print('Done ✅')

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/17565/fs/17485/fg/735584


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████| Rows 1002729/1002729 | Elapsed Time: 01:16 | Remaining Time: 00:00


Launching job: ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/17565/jobs/named/ranking_1_offline_fg_materialization/executions
Done ✅


---